# User Defined Functions

Apache Spark&trade; and Azure Databricks&reg; allow you to create your own User Defined Functions (UDFs) specific to the needs of your data.

-sandbox
### Custom Transformations with User Defined Functions

Spark's built-in functions provide a wide array of functionality, covering the vast majority of data transformation use cases. Often what differentiates strong Spark programmers is their ability to utilize built-in functions since Spark offers many highly optimized options to manipulate data. This matters for two reasons:<br><br>

- First, *built-in functions are finely tuned* so they run faster than less efficient code provided by the user.  
- Secondly, Spark (or, more specifically, Spark's optimization engine, the Catalyst Optimizer) knows the objective of built-in functions so it can *optimize the execution of your code by changing the execution order of your tasks.* 

In brief, use built-in functions whenever possible.

There are, however, many specific use cases not covered by built-in functions. **User Defined Functions (UDFs) are useful when you need to define logic specific to your use case and when you need to encapsulate that solution for reuse.** They should only be used when there is no clear way to accomplish a task using built-in functions.

UDFs are generally more performant in Scala than Python since for Python, Spark has to spin up a Python interpreter on every exector to run the function. This causes a substantial performance bottleneck due to communication accross the Py4J bridge (how the JVM interoperates with Python) and the slower nature of Python execution.

<div><img src="https://files.training.databricks.com/images/eLearning/ETL-Part-2/built-in-vs-udfs.png" style="height: 400px; margin: 20px"/></div>

### A Basic UDF

UDFs take a function or lambda and make it available for Spark to use.  Start by writing code in your language of choice that will operate on a single row of a single column in your DataFrame.

Run the cell below to mount the data.

In [5]:
%run "./Includes/Classroom-Setup"

Write a basic function that splits a string on an `e`.

In [7]:
def manual_split(x):
  return x.split("e")

manual_split("this is my example string")

Register the function as a UDF by designating the following:

* A name for access in Python (`manualSplitPythonUDF`)
* A name for access in SQL (`manualSplitSQLUDF`)
* The function itself (`manual_split`)
* The return type for the function (`StringType`)

In [9]:
from pyspark.sql.types import StringType

manualSplitPythonUDF = spark.udf.register("manualSplitSQLUDF", manual_split, StringType())

Create a dataframe of 100k values with a string to index. Do this by using a hash function.

In [11]:
from pyspark.sql.functions import sha1, rand
randomDF = (spark.range(1, 10000 * 10 * 10 * 10)
  .withColumn("random_value", rand(seed=10).cast("string"))
  .withColumn("hash", sha1("random_value"))
  .drop("random_value")
)

display(randomDF)

Apply the UDF by using it just like any other Spark function.

In [13]:
randomAugmentedDF = randomDF.select("*", manualSplitPythonUDF("hash").alias("augmented_col"))

display(randomAugmentedDF)

### DataFrame and SQL APIs

When you registered the UDF, it was named `manualSplitSQLUDF` for access in the SQL API. This gives us the same access to the UDF you had in the python DataFrames API.

Register `randomDF` to access it within SQL.

In [16]:
randomDF.createOrReplaceTempView("randomTable")

Now switch to the SQL API and use the same UDF.

In [18]:
%sql
SELECT id,
  hash,
  manualSplitSQLUDF(hash) as augmented_col
FROM
  randomTable

This is an easy way to generalize UDFs, allowing teams to share their code.

### Performance Tradeoffs

The performance of custom UDFs normally trail far behind built-in functions.  Take a look at this other example to compare built-in functions to custom UDFs.

Create a large DataFrame of random values, cache the result, and perform a `.count()` to actualize the cache.

In [22]:
from pyspark.sql.functions import col, rand

randomFloatsDF = (spark.range(0, 100 * 1000 * 1000)
  .withColumn("id", (col("id") / 1000).cast("integer"))
  .withColumn("random_float", rand())
)

randomFloatsDF.cache()
randomFloatsDF.count()

display(randomFloatsDF)

Register a new UDF that increments a column by 1.  Here, use a lambda instead of a function.

In [24]:
from pyspark.sql.types import FloatType
  
plusOneUDF = spark.udf.register("plusOneUDF", lambda x: x + 1, FloatType())

Compare the results using the `%timeit` function.  Run it a few times and examine the results.

In [26]:
%timeit randomFloatsDF.withColumn("incremented_float", plusOneUDF("random_float")).count()

In [27]:
%timeit randomFloatsDF.withColumn("incremented_float", col("random_float") + 1).count()

Which was faster, the UDF or the built-in functionality?  By how much?  This can differ based upon whether you work through this course in Scala (which is much faster) or Python.

## Exercise 1: Converting IP Addresses to Decimals

Write a UDF that translates an IPv4 address string (e.g. `123.123.123.123`) into a numeric value.

### Step 1: Create a Function

IP addresses pose challenges for efficient database lookups.  One way of dealing with this is to store an IP address in numerical form.  Write the function `IPConvert` that satisfies the following:

Input: IP address as a string (e.g. `123.123.123.123`)  
Output: an integer representation of the IP address (e.g. `2071690107`)

If the input string is `1.2.3.4`, think of it like `A.B.C.D` where A is 1, B is 2, etc. Solve this with the following steps:

&nbsp;&nbsp;&nbsp; (A x 256^3) + (B x 256^2) + (C x 256) + D <br>
&nbsp;&nbsp;&nbsp; (1 x 256^3) + (2 x 256^2) + (3 x 256) + 4 <br>
&nbsp;&nbsp;&nbsp; 116777216 + 131072 + 768 + 4 <br>
&nbsp;&nbsp;&nbsp; 16909060

Make a function to implement this.

In [31]:
# TODO
def IPConvert(IPString):
  FILL_IN

IPConvert("1.2.3.4") # should equal 16909060

In [32]:
# TEST - Run this cell to test your solution
dbTest("ET2-P-03-01-01", 16909060, IPConvert("1.2.3.4"))
dbTest("ET2-P-03-01-02", 168430090, IPConvert("10.10.10.10"))
dbTest("ET2-P-03-01-03", 386744599, IPConvert("23.13.65.23"))

print("Tests passed!")

### Step 2: Register a UDF

Register your function as `IPConvertUDF`.  Be sure to use `LongType` as your output type.

In [34]:
# TODO
IPConvertUDF = # FILL_IN

In [35]:
# TEST - Run this cell to test your solution
testDF = spark.createDataFrame((
  ("1.2.3.4", ),
  ("10.10.10.10", ),
  ("23.13.65.23", )
), ("ip",))
result = [i[0] for i in testDF.select(IPConvertUDF("ip")).collect()]

dbTest("ET2-P-03-02-01", 16909060, result[0])
dbTest("ET2-P-03-02-02", 168430090, result[1])
dbTest("ET2-P-03-02-03", 386744599, result[2])

print("Tests passed!")

### Step 3: Apply the UDF

Apply the UDF on the `IP` column of the DataFrame created below, creating the new column `parsedIP`.

In [37]:
# TODO
IPDF = spark.createDataFrame([["123.123.123.123"], ["1.2.3.4"], ["127.0.0.0"]], ['ip'])

IPDFWithParsedIP = # FILL_IN

display(IPDFWithParsedIP)

In [38]:
# TEST - Run this cell to test your solution
result2 = [i[1] for i in IPDFWithParsedIP.collect()]

dbTest("ET2-P-03-03-01", 2071690107, result2[0])
dbTest("ET2-P-03-03-02", 16909060, result2[1])
dbTest("ET2-P-03-03-03", 2130706432, result2[2])

print("Tests passed!")

## Review
**Question:** What are the performance tradeoffs between UDFs and built-in functions?  When should I use each?  
**Answer:** Built-in functions are normally faster than UDFs and should be used when possible.  UDFs should be used when specific use cases arise that aren't addressed by built-in functions.

**Question:** How can I use UDFs?  
**Answer:** UDFs can be used in any Spark API. They can be registered for use in SQL and can otherwise be used in Scala, Python, R, and Java.

**Question:** Why are built-in functions faster?  
**Answer:** Reasons include:
* The catalyst optimizer knows how to optimize built-in functions
* They are written in highly optimized Scala
* There is no serialization cost at the time of running a built-in function

**Question:** Can UDFs have multiple column inputs and outputs?  
**Answer:** Yes, UDFs can have multiple column inputs and multiple complex outputs. This is covered in the following lesson.

## Next Steps

Start the next lesson, [Advanced UDFs]($./04-Advanced-UDFs ).

## Additional Topics & Resources

**Q:** Where can I find out more about UDFs?  
**A:** Take a look at the <a href="https://docs.azuredatabricks.net/spark/latest/spark-sql/udf-scala.html#user-defined-functions-scala" target="_blank">Databricks documentation for more details</a>